일반화 성능도 있기 때문에 전체 데이터셋에 대해서 성능 평가를 해봐야 함

In [1]:
import pickle
with open('../models/BernoulliNB norm l30_r15.pkl', 'rb') as f:
    classifier = pickle.load(f)
    
from py.utils import load_data
head = 'l30_r15'
directory = '../data/'
x, y, x_words, vocabs = load_data(head, directory)

x shape = (15166, 2617)
y shape = (15166,)
# features = 2617
# L words = 15166


In [2]:
word_frequency = x.sum(axis=1)
word_frequency

matrix([[ 39],
        [215],
        [118],
        ..., 
        [129],
        [ 35],
        [ 36]])

In [3]:
def get_sample_l(ri, topk=5):
    nonzero = x[:,ri].nonzero()[0]
    base = min(50, len(nonzero)//2)
    return [x_words[l] for l in sorted(nonzero, key=lambda x:word_frequency[x,0], reverse=True)[base:base+topk]]

coefficient = log prob

==> exp(coefficient) = prob

In [4]:
import numpy as np
np.exp(classifier.class_log_prior_)

array([ 0.16523803,  0.83476197])

In [5]:
import numpy as np

base = 0
topk = 100
for r, coef in sorted(enumerate(classifier.coef_[0]), key=lambda x:x[1], reverse=True)[base :base + topk]:
    print('%s - %s (logp= %.3f, p= %.3f)' % (get_sample_l(r), vocabs[r], coef, np.exp(coef)))

['몸', '무엇', '얘기', '아버지', '모습'] - 의 (logp= -0.197, p= 0.821)
['의미', '몸', '무엇', '얘기', '아버지'] - 에 (logp= -0.272, p= 0.762)
['사회', '여기', '지금', '이야기', '안'] - 도 (logp= -0.567, p= 0.567)
['죽', '다음', '이상', '아이들', '삶'] - 을 (logp= -0.595, p= 0.551)
['책', '이들', '세상', '한국', '이름'] - 이 (logp= -0.612, p= 0.542)
['모습', '길', '죽', '다음', '이상'] - 은 (logp= -0.685, p= 0.504)
['밖', '쪽', '물', '적', '처음'] - 과 (logp= -0.809, p= 0.445)
['얘기', '아버지', '모습', '길', '다음'] - 에서 (logp= -0.865, p= 0.421)
['명', '주장', '입', '뜻', '발전'] - 으로 (logp= -1.016, p= 0.362)
['예', '역사', '시', '존재', '머리'] - 를 (logp= -1.018, p= 0.361)
['나가', '누구', '문화', '예', '역사'] - 가 (logp= -1.037, p= 0.354)
['삶', '뒤', '나라', '정부', '자기'] - 는 (logp= -1.075, p= 0.341)
['개', '남자', '생활', '학교', '자체'] - 로 (logp= -1.082, p= 0.339)
['예', '역사', '시', '존재', '머리'] - 와 (logp= -1.187, p= 0.305)
['아버지', '모습', '길', '다음', '이상'] - 인 (logp= -1.233, p= 0.291)
['내용', '책', '이들', '세상', '한국'] - 이나 (logp= -1.301, p= 0.272)
['데', '미국', '사랑', '관계', '자리'] - 하고 (logp= -1.426, p= 0.24

## BernoulliNB classifier code

    def _joint_log_likelihood(self, X):
        """Calculate the posterior log probability of the samples X"""
        check_is_fitted(self, "classes_")

        X = check_array(X, accept_sparse='csr')

        if self.binarize is not None:
            X = binarize(X, threshold=self.binarize)

        n_classes, n_features = self.feature_log_prob_.shape
        n_samples, n_features_X = X.shape

        if n_features_X != n_features:
            raise ValueError("Expected input with %d features, got %d instead"
                             % (n_features, n_features_X))

        neg_prob = np.log(1 - np.exp(self.feature_log_prob_))
        # Compute  neg_prob · (1 - X).T  as  ∑neg_prob - X · neg_prob
        jll = safe_sparse_dot(X, (self.feature_log_prob_ - neg_prob).T)
        jll += self.class_log_prior_ + neg_prob.sum(axis=1)

        return jll

### Bernoulli Naive Bayes 역시 infrequent feature가 low scale

classification rule: $ P(x_i \mid y) = P(i \mid y) x_i + (1 - P(i \mid y)) (1 - x_i) $

class y에서 feature i의 생성확률을 만들기 때문에 infrequent 한 feature가 작은 coefficient를 지님

In [6]:
np.exp(classifier.coef_[0,vocabs.index('해줌으로써')])

0.0011056705101879648